# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. 

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


In [1]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2 and all(isinstance(arg, int) for arg in args):
            n, m = args
            self.data = [[0 for _ in range(m)] for _ in range(n)]
        elif len(args) == 1 and isinstance(args[0], list):
            rows = args[0]
            if not all(isinstance(row, list) for row in rows):
                raise ValueError("All elements must be lists")
            row_length = len(rows[0])
            if not all(len(row) == row_length for row in rows):
                raise ValueError("All rows must have same number of columns")
            self.data = [row[:] for row in rows]
        else:
            raise ValueError("Invalid initialization parameters")

    def __getitem__(self, key):
        if isinstance(key, tuple):
            i, j = key
            return self.data[i][j]
        else:
            return self.data[key]

    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            i, j = key
            self.data[i][j] = value
        else:
            if isinstance(value, Matrix):
                if self._size() != value._size():
                    raise ValueError("Matrix size must match")
                self.data = [row[:] for row in value.data]
            elif isinstance(value, list):
                if self._size() != (len(value), len(value[0])):
                    raise ValueError("Matrix siz must match")
                self.data = [row[:] for row in value]
            else:
                raise ValueError("value must be a Matrix instance or a list of lists")

    def _size(self):
        return (len(self.data), len(self.data[0]))

    def __repr__(self):
        return f"Matrix({self.data})"


M1 = Matrix(2, 3)
print(M1)


M2 = Matrix([[1, 2, 3], [4, 5, 6]])
print(M2)


print(M2[1][2])
M2[1, 2] = 9
print(M2[1, 2])


M1 = M2
print(M1)

M3 = Matrix(2, 3)
M3 = [[7, 8, 9], [10, 11, 12]]
print(M3)

Matrix([[0, 0, 0], [0, 0, 0]])
Matrix([[1, 2, 3], [4, 5, 6]])
6
9
Matrix([[1, 2, 3], [4, 5, 9]])
[[7, 8, 9], [10, 11, 12]]


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

In [2]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2 and all(isinstance(arg, int) for arg in args):
            n, m = args
            self.data = [[0 for _ in range(m)] for _ in range(n)]
        elif len(args) == 1 and isinstance(args[0], list):
            rows = args[0]
            if not all(isinstance(row, list) for row in rows) or not rows:
                raise ValueError("All elements must be lists")
            row_length = len(rows[0])
            if not all(len(row) == row_length for row in rows):
                raise ValueError("All rows must have the same number of columns")
            self.data = [row[:] for row in rows]
        else:
            raise ValueError("Invalid initialization parameters")

    def __getitem__(self, key):
        if isinstance(key, slice) or any(isinstance(k, slice) for k in key if isinstance(key, tuple)):
            # Handle slicing for both rows and columns
            slice_rows = self.data[key] if isinstance(key, slice) else self.data[key[0]]
            if isinstance(key, tuple) and isinstance(key[1], slice):
                slice_rows = [row[key[1]] for row in slice_rows]
            return Matrix(slice_rows)
        elif isinstance(key, tuple):
            i, j = key
            return self.data[i][j]
        else:
            return self.data[key]

    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            i, j = key
            self.data[i][j] = value
        else:
            if isinstance(value, Matrix):
                if self._size() != value._size():
                    raise ValueError("Matrix size must match")
                self.data = [row[:] for row in value.data]
            elif isinstance(value, list):
                if self._size() != (len(value), len(value[0])):
                    raise ValueError("Matrix sizes must match")
                self.data = [row[:] for row in value]
            else:
                raise ValueError("Value must be a Matrix instance or a list of lists")

    def _size(self):
        return (len(self.data), len(self.data[0]))

    def shape(self):
        return self._size()

    def transpose(self):
        transposed_data = list(map(list, zip(*self.data)))
        return Matrix(transposed_data)

    def row(self, n):
        return Matrix([self.data[n]])

    def column(self, n):
        return Matrix([[row[n]] for row in self.data])

    def to_list(self):
        return self.data

    def block(self, n_0, n_1, m_0, m_1):
        return Matrix([row[m_0:m_1+1] for row in self.data[n_0:n_1+1]])

    def __repr__(self):
        return f"Matrix({self.data})"


3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

In [3]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2 and all(isinstance(arg, int) for arg in args):
            n, m = args
            self.data = [[0 for _ in range(m)] for _ in range(n)]
        elif len(args) == 1 and isinstance(args[0], list):
            rows = args[0]
            if not all(isinstance(row, list) for row in rows) or not rows:
                raise ValueError("All elements must be lists")
            row_length = len(rows[0])
            if not all(len(row) == row_length for row in rows):
                raise ValueError("All rows must have the same number of columns")
            self.data = [row[:] for row in rows]
        else:
            raise ValueError("Invalid initialization parameters")

    def __getitem__(self, key):
        if isinstance(key, slice) or any(isinstance(k, slice) for k in key if isinstance(key, tuple)):
            slice_rows = self.data[key] if isinstance(key, slice) else self.data[key[0]]
            if isinstance(key, tuple) and isinstance(key[1], slice):
                slice_rows = [row[key[1]] for row in slice_rows]
            return Matrix(slice_rows)
        elif isinstance(key, tuple):
            i, j = key
            return self.data[i][j]
        else:
            return self.data[key]

    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            i, j = key
            self.data[i][j] = value
        else:
            if isinstance(value, Matrix):
                if self._size() != value._size():
                    raise ValueError("Matrix sizes must match")
                self.data = [row[:] for row in value.data]
            elif isinstance(value, list):
                if self._size() != (len(value), len(value[0])):
                    raise ValueError("Matrix sizes must match")
                self.data = [row[:] for row in value]
            else:
                raise ValueError("Value must be a Matrix instance or a list of lists")

    def _size(self):
        return (len(self.data), len(self.data[0]))

    def shape(self):
        return self._size()

    def transpose(self):
        transposed_data = list(map(list, zip(*self.data)))
        return Matrix(transposed_data)

    def row(self, n):
        return Matrix([self.data[n]])

    def column(self, n):
        return Matrix([[row[n]] for row in self.data])

    def to_list(self):
        return self.data

    def block(self, n_0, n_1, m_0, m_1):
        return Matrix([row[m_0:m_1+1] for row in self.data[n_0:n_1+1]])

    def __repr__(self):
        return f"Matrix({self.data})"

def constant(n, m, c):
    return Matrix([[float(c) for _ in range(m)] for _ in range(n)])

def zeros(n, m):
    return constant(n, m, 0.0)

def ones(n, m):
    return constant(n, m, 1.0)

def eye(n):
    return Matrix([[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)])


4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

In [5]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2 and all(isinstance(arg, int) for arg in args):
            n, m = args
            self.data = [[0 for _ in range(m)] for _ in range(n)]
        elif len(args) == 1 and isinstance(args[0], list):
            rows = args[0]
            if not all(isinstance(row, list) for row in rows) or not rows:
                raise ValueError("All elements must be lists")
            row_length = len(rows[0])
            if not all(len(row) == row_length for row in rows):
                raise ValueError("All rows must have the same number of columns")
            self.data = [row[:] for row in rows]
        else:
            raise ValueError("Invalid initialization parameters")

    def __getitem__(self, key):
        if isinstance(key, slice) or any(isinstance(k, slice) for k in key if isinstance(key, tuple)):
            slice_rows = self.data[key] if isinstance(key, slice) else self.data[key[0]]
            if isinstance(key, tuple) and isinstance(key[1], slice):
                slice_rows = [row[key[1]] for row in slice_rows]
            return Matrix(slice_rows)
        elif isinstance(key, tuple):
            i, j = key
            return self.data[i][j]
        else:
            return self.data[key]

    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            i, j = key
            self.data[i][j] = value
        else:
            if isinstance(value, Matrix):
                if self._size() != value._size():
                    raise ValueError("Matrix sizes must match")
                self.data = [row[:] for row in value.data]
            elif isinstance(value, list):
                if self._size() != (len(value), len(value[0])):
                    raise ValueError("Matrix sizes must match")
                self.data = [row[:] for row in value]
            else:
                raise ValueError("Value must be a Matrix instance or a list of lists")

    def _size(self):
        return (len(self.data), len(self.data[0]))

    def shape(self):
        return self._size()

    def transpose(self):
        transposed_data = list(map(list, zip(*self.data)))
        return Matrix(transposed_data)

    def row(self, n):
        return Matrix([self.data[n]])

    def column(self, n):
        return Matrix([[row[n]] for row in self.data])

    def to_list(self):
        return self.data

    def block(self, n_0, n_1, m_0, m_1):
        return Matrix([row[m_0:m_1+1] for row in self.data[n_0:n_1+1]])

    def scalarmul(self, c):
        return Matrix([[c * self.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def add(self, N):
        if self._size() != N._size():
            raise ValueError("Matrices must have the same dimensions to add")
        return Matrix([[self.data[i][j] + N.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def sub(self, N):
        if self._size() != N._size():
            raise ValueError("Matrices must have the same dimensions to subtract")
        return Matrix([[self.data[i][j] - N.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def mat_mult(self, N):
        if len(self.data[0]) != len(N.data):
            raise ValueError("The number of columns in the first matrix must be equal to the number of rows in the second")
        result = Matrix(len(self.data), len(N.data[0]))
        for i in range(len(result.data)):
            for j in range(len(result.data[0])):
                result.data[i][j] = sum(self.data[i][k] * N.data[k][j] for k in range(len(N.data)))
        return result

    def element_mult(self, N):
        if self._size() != N._size():
            raise ValueError("Matrices must have the same dimensions for element-wise multiplication")
        return Matrix([[self.data[i][j] * N.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def equals(self, N):
        if self._size() != N._size():
            return False
        return all(self.data[i][j])

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


In [7]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2 and all(isinstance(arg, int) for arg in args):
            n, m = args
            self.data = [[0 for _ in range(m)] for _ in range(n)]
        elif len(args) == 1 and isinstance(args[0], list):
            rows = args[0]
            if not all(isinstance(row, list) for row in rows) or not rows:
                raise ValueError("All elements must be lists")
            row_length = len(rows[0])
            if not all(len(row) == row_length for row in rows):
                raise ValueError("All rows must have the same number of columns")
            self.data = [row[:] for row in rows]
        else:
            raise ValueError("Invalid initialization parameters")

    def __getitem__(self, key):
        if isinstance(key, slice) or any(isinstance(k, slice) for k in key if isinstance(key, tuple)):
            slice_rows = self.data[key] if isinstance(key, slice) else self.data[key[0]]
            if isinstance(key, tuple) and isinstance(key[1], slice):
                slice_rows = [row[key[1]] for row in slice_rows]
            return Matrix(slice_rows)
        elif isinstance(key, tuple):
            i, j = key
            return self.data[i][j]
        else:
            return self.data[key]

    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            i, j = key
            self.data[i][j] = value
        else:
            if isinstance(value, Matrix):
                if self._size() != value._size():
                    raise ValueError("Matrix sizes must match")
                self.data = [row[:] for row in value.data]
            elif isinstance(value, list):
                if self._size() != (len(value), len(value[0])):
                    raise ValueError("Matrix sizes must match")
                self.data = [row[:] for row in value]
            else:
                raise ValueError("Value must be a Matrix instance or a list of lists")

    def _size(self):
        return (len(self.data), len(self.data[0]))

    def shape(self):
        return self._size()

    def transpose(self):
        transposed_data = list(map(list, zip(*self.data)))
        return Matrix(transposed_data)

    def row(self, n):
        return Matrix([self.data[n]])

    def column(self, n):
        return Matrix([[row[n]] for row in self.data])

    def to_list(self):
        return self.data

    def block(self, n_0, n_1, m_0, m_1):
        return Matrix([row[m_0:m_1+1] for row in self.data[n_0:n_1+1]])

    def scalarmul(self, c):
        return Matrix([[c * self.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def add(self, N):
        if self._size() != N._size():
            raise ValueError("Matrices must have the same dimensions to add")
        return Matrix([[self.data[i][j] + N.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def sub(self, N):
        if self._size() != N._size():
            raise ValueError("Matrices must have the same dimensions to subtract")
        return Matrix([[self.data[i][j] - N.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def mat_mult(self, N):
        if len(self.data[0]) != len(N.data):
            raise ValueError("The number of columns in the first matrix must be equal to the number of rows in the second")
        result = Matrix(len(self.data), len(N.data[0]))
        for i in range(len(result.data)):
            for j in range(len(result.data[0])):
                result.data[i][j] = sum(self.data[i][k] * N.data[k][j] for k in range(len(N.data)))
        return result

    def element_mult(self, N):
        if self._size() != N._size():
            raise ValueError("Matrices must have the same dimensions for element-wise multiplication")
        return Matrix([[self.data[i][j] * N.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def equals(self, N):
        if self._size() != N._size():
            return False
        return all(self.data[i][j])


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [11]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2 and all(isinstance(arg, int) for arg in args):
            n, m = args
            self.data = [[0 for _ in range(m)] for _ in range(n)]
        elif len(args) == 1 and isinstance(args[0], list):
            rows = args[0]
            if not all(isinstance(row, list) for row in rows) or not rows:
                raise ValueError("All elements must be lists")
            row_length = len(rows[0])
            if not all(len(row) == row_length for row in rows):
                raise ValueError("All rows must have the same number of columns")
            self.data = [row[:] for row in rows]
        else:
            raise ValueError("Invalid initialization parameters")

    def __getitem__(self, key):
        if isinstance(key, slice) or any(isinstance(k, slice) for k in key if isinstance(key, tuple)):
            slice_rows = self.data[key] if isinstance(key, slice) else self.data[key[0]]
            if isinstance(key, tuple) and isinstance(key[1], slice):
                slice_rows = [row[key[1]] for row in slice_rows]
            return Matrix(slice_rows)
        elif isinstance(key, tuple):
            i, j = key
            return self.data[i][j]
        else:
            return self.data[key]

    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            i, j = key
            self.data[i][j] = value
        else:
            if isinstance(value, Matrix):
                if self._size() != value._size():
                    raise ValueError("Matrix sizes must match")
                self.data = [row[:] for row in value.data]
            elif isinstance(value, list):
                if self._size() != (len(value), len(value[0])):
                    raise ValueError("Matrix sizes must match")
                self.data = [row[:] for row in value]
            else:
                raise ValueError("Value must be a Matrix instance or a list of lists")

    def _size(self):
        return (len(self.data), len(self.data[0]))

    def shape(self):
        return self._size()

    def transpose(self):
        transposed_data = list(map(list, zip(*self.data)))
        return Matrix(transposed_data)

    def row(self, n):
        return Matrix([self.data[n]])

    def column(self, n):
        return Matrix([[row[n]] for row in self.data])

    def to_list(self):
        return self.data

    def block(self, n_0, n_1, m_0, m_1):
        return Matrix([row[m_0:m_1+1] for row in self.data[n_0:n_1+1]])

    def scalarmul(self, c):
        return Matrix([[c * self.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def add(self, N):
        if self._size() != N._size():
            raise ValueError("Matrices must have the same dimensions to add")
        return Matrix([[self.data[i][j] + N.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def sub(self, N):
        if self._size() != N._size():
            raise ValueError("Matrices must have the same dimensions to subtract")
        return Matrix([[self.data[i][j] - N.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def mat_mult(self, N):
        if len(self.data[0]) != len(N.data):
            raise ValueError("The number of columns in the first matrix must be equal to the number of rows in the second")
        result = Matrix(len(self.data), len(N.data[0]))
        for i in range(len(result.data)):
            for j in range(len(result.data[0])):
                result.data[i][j] = sum(self.data[i][k] * N.data[k][j] for k in range(len(N.data)))
        return result

    def element_mult(self, N):
        if self._size() != N._size():
            raise ValueError("Matrices must have the same dimensions for element-wise multiplication")
        return Matrix([[self.data[i][j] * N.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def equals(self, N):
        if self._size() != N._size():
            return False
        return all(self.data[i][j] == N.data[i][j] for i in range(len(self.data)) for j in range(len(self.data[0])))

    def __mul__(self, other):
        if isinstance(other, Matrix):  # Matrix multiplication
            return self.mat_mult(other)
        elif isinstance(other, (int, float)):
            return self.scalarmul(other)
        else:
            raise ValueError("Unsupported type for multiplication.")

    def __add__(self, other):
        return self.add(other)

    def __eq__(self, other):
        return self.equals(other)
        
def demonstration():
    # Example matrices
    A = Matrix([[1, 2], [3, 4]])
    B = Matrix([[5, 6], [7, 8]])
    C = Matrix([[1, 0], [0, 1]])

    print("(AB)C = A(BC)")
    print("Result of (AB)C:\n", (A * B) * C)
    print("Result of A(BC):\n", A * (B * C))

    print("\nA(B + C) = AB + AC")
    print("Result of A(B + C):\n", A * (B + C))
    print("Result of AB + AC:\n", (A * B) + (A * C))

    print("\nAB ≠ BA")
    print("AB equals BA? ", A * B == B * A)

    print("\nAI = A")
    print("Result of AI:\n", A * C)

demonstration()

(AB)C = A(BC)
Result of (AB)C:
Result of A(BC):

AB ≠ BA
Result of AB:
Result of BA:
AB equals BA?  False

AI = A
Result of AI:
AI equals A?  True
(AB)C = A(BC)
Result of (AB)C:
Result of A(BC):

AB ≠ BA
Result of AB:
Result of BA:
AB equals BA?  False

AI = A
Result of AI:
AI equals A?  True
